In [ ]:
import Pkg; Pkg.activate(@__DIR__); Pkg.instantiate();

In [ ]:
using GLMakie
using LinearAlgebra
# using Symbolics
using ForwardDiff
using BenchmarkTools
using Colors

GLMakie.activate!(inline=false)
P2_f64 = Point{2, Float64}
P3_f64 = Point{3, Float64}

In [690]:

function divergence(f, vec::Vector{P2_f64})
    return sum.(diag.(ForwardDiff.jacobian.(x ->f(x[1],x[2]), vec)))
end

function curl_2d(f, vec::Vector{P2_f64})
    jac_vec = ForwardDiff.jacobian.(x ->f(x[1],x[2]), vec)
    curl_vec = Float64[]
    
    for jac in jac_vec
        curl = jac[2,1] - jac[1,2]
        push!(curl_vec, curl)
    end

    return curl_vec 
end

function curl_3d(f, vec::Vector{P3_f64})
    jac_vec = ForwardDiff.jacobian.(x ->f(x[1],x[2],x[3]), vec)
    curl_vec = P3_f64[]
    
    for jac in jac_vec
        (∂f₁∂x, ∂f₁∂y, ∂f₁∂z, 
        ∂f₂∂x, ∂f₂∂y, ∂f₂∂z,
        ∂f₃∂x, ∂f₃∂y, ∂f₃∂z) = transpose(jac) # transpose for correct ordering

        # @show jac
    
        curl_x =   ∂f₃∂y - ∂f₂∂z
        curl_y = -(∂f₃∂x - ∂f₁∂z)
        curl_z =   ∂f₂∂x - ∂f₁∂y
        
        curl = P3_f64(curl_x, curl_y, curl_z)
        push!(curl_vec, curl)
    end

    return curl_vec 
end
    

curl_3d (generic function with 2 methods)

# Divergence

### Vector field

In [ ]:
let

fig = Figure(resolution = (800, 800))
Axis(fig[1, 1], backgroundcolor = "black" , title = "Vector field")

x_vec = LinRange(-1, 1, 20)
y_vec = LinRange(-1, 1, 20)

f₁(x,y) = x*y
f₂(x,y) = y^2-x^2
    
f(x,y) = [f₁(x,y), f₂(x,y)]

input_vec =  P2_f64[];
f_vec = P2_f64[];

for x in x_vec
    for y in y_vec
        push!(input_vec, P2_f64(x,y))
        push!(f_vec, f(x,y))
    end
end


strength = sqrt.(norm.(f_vec))

# @timev Point2f.(input_vec)
arrows!(input_vec, f_vec, arrowsize = 20, lengthscale = 0.3,
    arrowcolor = strength, linecolor = strength, linewidth = 4)
display(fig)

end

### Divergence Computation

In [ ]:
let

fig = Figure(resolution = (1600, 800))
ax1 = Axis(fig[1, 1], backgroundcolor = "black" , title = "Vector field")
ax2 = Axis(fig[1, 2], backgroundcolor = "black" , title = "Divergence Colormap")

x_vec = LinRange(-1, 1, 20)
y_vec = LinRange(-1, 1, 20)

f₁(x,y) = x*y
f₂(x,y) = y^2 - x^2
    
f(x,y) = [f₁(x,y), f₂(x,y)]
    
input_vec = P2_f64[];
f_vec = P2_f64[];

for x in x_vec
    for y in y_vec
        push!(input_vec, Point2f(x,y))'
        push!(f_vec, f(x,y));
    end
end

strength = sqrt.(norm.(f_vec))

# @timev Point2f.(input_vec)
arrows!(ax1, input_vec, f_vec, arrowsize = 20, lengthscale = 0.3,
    arrowcolor = strength, linecolor = strength, linewidth = 4)

# compute divergence
divergence_vec = divergence(f, input_vec);
    
hm = heatmap!(ax2, [vec[1] for vec in input_vec] , [vec[2] for vec in input_vec], divergence_vec,  colormap= :deep)
Colorbar(fig[:, end+1], hm)

display(fig)
end

# Curl

### 2D Vector field

In [ ]:
let

fig = Figure(resolution = (800, 800))
Axis(fig[1, 1], backgroundcolor = "black" , title = "Vector field")

x_vec = LinRange(-4, 4, 20)
y_vec = LinRange(-4, 4, 20)
    
f₁(x,y) = y^3 - 9*y
f₂(x,y) = x^3 - 9*x
    
f(x,y) = [f₁(x,y), f₂(x,y)]

input_vec =  P2_f64[];
f_vec = P2_f64[];

for x in x_vec
    for y in y_vec
        push!(input_vec, P2_f64(x,y))
        push!(f_vec, f(x,y))
    end
end


strength = sqrt.(norm.(f_vec))

# @timev Point2f.(input_vec)
arrows!(input_vec, f_vec, arrowsize = 20, lengthscale = 0.01,
    arrowcolor = strength, linecolor = strength, linewidth = 4)
display(fig)

end

### 2D Animation

In [ ]:
let

fig = Figure(resolution = (800, 800))
ax1 = Axis(fig[1, 1], backgroundcolor = "black" , title = "Vector field")

x_vec = LinRange(-4, 4, 20)
y_vec = LinRange(-4, 4, 20)

f₁(x,y) = y^3 - 9*y
f₂(x,y) = x^3 - 9*x
    
f(x,y) = [f₁(x,y), f₂(x,y)]
    
input_vec = Point2f[];
f_vec = Point2f[];

for x in x_vec
    for y in y_vec
        push!(input_vec, Point2f(x,y))
        push!(f_vec, f(x,y))
    end
end

strength = sqrt.(norm.(f_vec))

arrows!(input_vec, f_vec, arrowsize = 20, lengthscale = 0.01,
    arrowcolor = strength, linecolor = strength, linewidth = 4)
display(fig)

# Plot fluid particle 
X_initial = input_vec[50]

X = Observable{Point2f}(X_initial)

scatter!(ax1, X, markersize = 60)

tspan = (0.0,5.0)
dt= 0.01

for t in tspan[1]:dt:tspan[2]
    velocity = f(X[][1], X[][2])
    X[] = X[] + (velocity*dt)
    sleep(dt)
end

end

### 2D Curl Computation 

In [ ]:
let

fig = Figure(resolution = (1600, 800))
ax1 = Axis(fig[1, 1], backgroundcolor = "black" , title = "Vector field")
ax2 = Axis(fig[1, 2], title = "Curl of Vector field")

limit = 4
x_vec = LinRange(-limit, limit, 20)
y_vec = LinRange(-limit, limit, 20)
    
f₁(x,y) = y^3 - 9*y
f₂(x,y) = x^3 - 9*x
    
f(x,y) = [f₁(x,y), f₂(x,y)]

input_vec =  P2_f64[];
f_vec = P2_f64[];

for x in x_vec
    for y in y_vec
        push!(input_vec, P2_f64(x,y))
        push!(f_vec, f(x,y))
    end
end


strength = sqrt.(norm.(f_vec))

# @timev Point2f.(input_vec)
arrows!(ax1, input_vec, f_vec, arrowsize = 20, lengthscale = 0.01,
    arrowcolor = strength, linecolor = strength, linewidth = 4)

    
curl_vec = curl_2d(f, input_vec);

# for transparent heatmp:  colormap=RGBAf.(Colors.color.(to_colormap(:deep)), 0.5)
hm = heatmap!(ax2, [vec[1] for vec in input_vec] , [vec[2] for vec in input_vec], curl_vec,  colormap=:deep)
Colorbar(fig[:, end+1], hm)

display(fig)    
end

### 3D Vector field

In [ ]:
let

fig = Figure()


ax1 = Axis3(fig[1, 1], aspect = :equal, title = "3D Vector field",
        #limits =(ax_low, ax_high,ax_low, ax_high,ax_low, ax_high,
        xlabel="X", ylabel="Y", zlabel="Z",
        xlabelsize=30, ylabelsize=30, zlabelsize=30
    )

limit = 4
x_vec = LinRange(-limit, limit, 20)
y_vec = LinRange(-limit, limit, 20)
z_vec = LinRange(-limit, limit, 20)

f₁(x,y,z) = y^3 - 9*y
f₂(x,y,z) = x^3 - 9*x
f₃(x,y,z) = 0
    
f(x,y,z) = [f₁(x,y,z), f₂(x,y,z), f₃(x,y,z)]

input_vec =  P3_f64[];
f_vec = P3_f64[];

for x in x_vec
    for y in y_vec
        for z in z_vec
            push!(input_vec, P3_f64(x,y,z))
            push!(f_vec, f(x,y,z))
        end
    end
end
        

arrow_scale = 0.02
f_vec_scaled = map(p -> arrow_scale * Vec3f(p[1], p[2], p[3]), f_vec)
    
arrows!(
    input_vec,  f_vec_scaled, fxaa=true, # turn on anti-aliasing
    linecolor = :gray, arrowcolor = :black,
    linewidth = 0.01, arrowsize = Vec3f(0.03, 0.03, 0.06),
    align = :center
)

println(sizeof(input_vec))
display(fig)
end

### 3D curl computation

In [950]:
let

fig = Figure()


ax1 = Axis3(fig[1, 1], aspect = :equal, title = "3D Vector field",
        #limits =(ax_low, ax_high,ax_low, ax_high,ax_low, ax_high,
        xlabel="X", ylabel="Y", zlabel="Z",
        xlabelsize=30, ylabelsize=30, zlabelsize=30
    )

limit = 4
x_vec = LinRange(-limit, limit, 10)
y_vec = LinRange(-limit, limit, 10)
z_vec = LinRange(-limit, limit, 10)

f₁(x,y,z) = y^3 - 9*y
f₂(x,y,z) = x^3 - 9*x
f₃(x,y,z) = 0
    
f(x,y,z) = [f₁(x,y,z), f₂(x,y,z), f₃(x,y,z)]

input_vec =  P3_f64[];
f_vec = P3_f64[];

for x in x_vec
    for y in y_vec
        for z in z_vec
            push!(input_vec, P3_f64(x,y,z))
            push!(f_vec, f(x,y,z))
        end
    end
end
        

curl_vec = curl_3d(f, input_vec);

# plot vector field
arrow_scale = 0.02
arrows!(
    input_vec,  f_vec*arrow_scale , fxaa=true, # turn on anti-aliasing
    linecolor = :gray, arrowcolor = :black,
    linewidth = 0.01, arrowsize = Vec3f(0.1, 0.1, 0.3),
    align = :center
)

# plot curl
arrow_scale = 0.01
arrows!(
    input_vec,  curl_vec*arrow_scale , fxaa=true, # turn on anti-aliasing
    linecolor = :gray, arrowcolor = :green,
    linewidth = 0.01, arrowsize = Vec3f(0.2, 0.2, 0.4),
    align = :center
)
    
display(fig)
end

GLMakie.Screen(...)

# Gradient

In [982]:
let 

fig = Figure()
ax1 = Axis3(fig[1, 1], aspect = :equal, title = "3D Vector field",
        #limits =(ax_low, ax_high,ax_low, ax_high,ax_low, ax_high,
        xlabel="X", ylabel="Y", zlabel="Z",
        xlabelsize=30, ylabelsize=30, zlabelsize=30
    )
    
x_vec =y_vec = LinRange(-5, 5, 20)

f(x,y) = x^2 + y^2
z_vec = [f(x,y) for x in x_vec, y in y_vec]

zmin, zmax = minimum(z_vec), maximum(z_vec)

# plot surface
surface!(ax1, x_vec, y_vec, z_vec, colormap = :viridis, colorrange = (zmin, zmax), transparency = true)

# plot wireframe over surface    
wireframe!(ax1, x_vec, y_vec, z_vec; overdraw = true, transparency = true, color = (:black, 0.2))

# plot contour plot underneath surface
contour!(ax1, x_vec, y_vec, z_vec; levels = 10, colormap = :viridis, linewidth = 2,
    colorrange = (zmin, zmax))
    
# # Compute derivative
f(var) = f(var[1],var[2])

grad_vec =  P2_f64[];
input_vec =  P2_f64[];
    
# for x in x_vec
#     for y in y_vec
#         grad = ForwardDiff.gradient(f, [x y])
#         push!(grad_vec, grad)
#         push!(input_vec, P2_f64(x,y))
#     end
# end

strength = sqrt.(norm.(grad_vec))

# plot the gradient vectors
arrows!(ax1, input_vec, grad_vec, arrowsize = 22, lengthscale = 0.05,
    arrowcolor = strength, linecolor = strength, linewidth = 4,  overdraw = true, transparency = true)

display(fig)       

end

GLMakie.Screen(...)

## DIrectional derivative 

In [1000]:
let 

fig = Figure()
ax1 = Axis3(fig[1, 1], aspect = :equal, title = "3D Vector field",
        #limits =(ax_low, ax_high,ax_low, ax_high,ax_low, ax_high,
        xlabel="X", ylabel="Y", zlabel="Z",
        xlabelsize=30, ylabelsize=30, zlabelsize=30
    )
    
x_vec =y_vec = LinRange(-20, 20, 10)
f(x,y) = x^2*y
z_vec = [f(x,y) for x in x_vec, y in y_vec]

zmin, zmax = minimum(z_vec), maximum(z_vec)

# plot surface
surface!(ax1, x_vec, y_vec, z_vec, colormap = :viridis, colorrange = (zmin, zmax), transparency = true)

# plot wireframe over surface    
wireframe!(ax1, x_vec, y_vec, z_vec; overdraw = true, transparency = true, color = (:black, 0.2))

    
# Draw intersecting plane
x_vec = z_vec = LinRange(-20, 20, 10)


# plot surface
surface!(ax1, x_vec, y_vec, z_vec, colormap = :viridis, transparency = true, overdraw = true,)
    
display(fig)   
    
end

GLMakie.Screen(...)